# Worksheet 08

Name:  Ivanna M
UID: U69469925

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.708619598341939, 5.024427794026485, 5.235113248861341, 6.442723348419195, 5.856582360864448, 4.762819880108156, 5.985208393294818, 6.380606088795176, 5.684483599346628, 4.920183863026301]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [8]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[6.912206117086036, 8.568105883391004, 9.455632315923163, 8.815923156009507, 6.714533948909612, 9.276770297514524, 7.556072678055267, 9.679244229300338, 8.47123832810856, 7.726731821707084]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [9]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.726731821707084, 6.442723348419195, 5.856582360864448, 4.762819880108156, 8.47123832810856, 5.985208393294818, 6.0265890314046855, 9.679244229300338, 7.556072678055267, 9.276770297514524]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Number of components = 2
- Weights: probability associated with each component that a datapoint belongs to said component.
- mean of each component
- variance

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [10]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2)/(len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2)  ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.726731821707084, 8.47123832810856, 9.679244229300338, 7.556072678055267, 9.276770297514524]
[6.442723348419195, 5.856582360864448, 4.762819880108156, 5.985208393294818, 6.0265890314046855]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 8.542011470937155,  mean_2 = 5.814784602818261
var_1 = 0.6949867854863248,  var_2 = 7.7530842467816585


C:\Users\ivann\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Most of the parameters are close to the true values, except variance.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [11]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]*prob_s[0]/prob_x )
    prob_s1_x.append( pdf_i[1]*prob_s[1]/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.726731821707084
probability of observing that point if it came from cluster 0 =  0.2884763677211606
probability of observing that point if it came from cluster 1 =  0.04991487570886329
point =  6.442723348419195
probability of observing that point if it came from cluster 0 =  0.005993161310533134
probability of observing that point if it came from cluster 1 =  0.05128745356267344
point =  5.856582360864448
probability of observing that point if it came from cluster 0 =  0.00032871316170918674
probability of observing that point if it came from cluster 1 =  0.05145519773744122
point =  4.762819880108156
probability of observing that point if it came from cluster 0 =  2.1776454595624525e-07
probability of observing that point if it came from cluster 1 =  0.05098446730213555
point =  8.47123832810856
probability of observing that point if it came from cluster 0 =  0.5710599138115292
probability of observing that point if it came from cluster 1 =  0.04852250526218404
point =  5.


7.726731821707084
Probability of coming from S_1 = 0.852493595274769
Probability of coming from S_2 = 0.14750640472523108
Point is likely in cluster 0.

6.442723348419195
Probability of coming from S_1 = 0.10462808969141285
Probability of coming from S_2 = 0.8953719103085872
Point is likely in cluster 1.


5.856582360864448
Probability of coming from S_1 = 0.006347785557358893
Probability of coming from S_2 = 0.9936522144426411
Point is likely in cluster 1.

4.762819880108156
Probability of coming from S_1 = 4.271175661307642e-06
Probability of coming from S_2 = 0.9999957288243386
Point is likely in cluster 1.

8.47123832810856
Probability of coming from S_1 = 0.9216851483056506
Probability of coming from S_2 = 0.07831485169434932
Point is likely in cluster 0.

5.985208393294818
Probability of coming from S_1 = 0.012679057372525853
Probability of coming from S_2 = 0.9873209426274742
Point is likely in cluster 1.

6.0265890314046855
Probability of coming from S_1 = 0.015709807914952596
Probability of coming from S_2 = 0.9842901920850473
Point is likely in cluster 1.

9.679244229300338
Probability of coming from S_1 = 0.7680575537759298
Probability of coming from S_2 = 0.23194244622407015
Point is likely in cluster 0.

7.556072678055267
Probability of coming from S_1 = 0.8070413297647252
Probability of coming from S_2 = 0.19295867023527483
Point is likely in cluster 0.

9.276770297514524
Probability of coming from S_1 = 0.8757498090370002
Probability of coming from S_2 = 0.12425019096299973
Point is likely in cluster 0.

The points that lie between 7-10 are classified together, which makes sense since mean_1 = 8.542011470937155 and mean_2 = 5.814784602818261. 


f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [13]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), 
        sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.43643964478699865,  P(S_2) = 0.5635603552130014
mean_1 = 8.462352963279683,  mean_2 = 6.184061180464873
var_1 = 0.798141402017749,  var_2 = 1.3123166959221793



The intra-cluster variance values are a lot lower now. There is also a higher P(S_2)

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [26]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0]*prob_s[0]/prob_x )
    prob_s1_x.append( pdf_i[1]*prob_s[1]/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.726731821707084
probability of observing that point if it came from cluster 0 =  0.3268660921603573
probability of observing that point if it came from cluster 1 =  0.15233539353109152
point =  6.442723348419195
probability of observing that point if it came from cluster 0 =  0.020343962509176867
probability of observing that point if it came from cluster 1 =  0.2981503270017867
point =  5.856582360864448
probability of observing that point if it came from cluster 0 =  0.00242257892931397
probability of observing that point if it came from cluster 1 =  0.29467909917286195
point =  4.762819880108156
probability of observing that point if it came from cluster 0 =  1.0800272528413927e-05
probability of observing that point if it came from cluster 1 =  0.1691145484488235
point =  8.47123832810856
probability of observing that point if it came from cluster 0 =  0.4998081278339895
probability of observing that point if it came from cluster 1 =  0.06656984374431095
point =  5.98520

The probabilities match the real position of the points more closely.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [25]:
assignments = []

for i in zip(data, prob_s0_x):
    point = i[0]
    p = i[1]
    if random.random() <= p:
        assignments.append((point,0))
    else:
        assignments.append((point, 1))
print(assignments)

[(7.726731821707084, 0), (6.442723348419195, 1), (5.856582360864448, 1), (4.762819880108156, 1), (8.47123832810856, 1), (5.985208393294818, 1), (6.0265890314046855, 1), (9.679244229300338, 0), (7.556072678055267, 1), (9.276770297514524, 0)]
